# Imports

In [1]:
import os
import sys
import math
import time
import argparse
import numpy as np

from PIL import Image

from datetime import datetime

import dataGenerator
import logly

import torch
import torch.utils
import torch.utils.data
import torch.nn.parallel
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.optim import lr_scheduler

import torchvision
import torchvision.utils as vutils
from torchvision.transforms import ToPILImage
import torchvision.transforms as transforms
from torchvision import datasets, models, transforms

# Define DataLoader

In [2]:
#Set Values
show_live_loss = False
newTraining = True
save_masks = False
print_loss = True
output_tensors = False
EPOCHS = 5
iteration = "-14"

# default values
NUM_CLASSES = 1
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
batchSize = 50
#val_batch_size = 100
num_batch = 2000 #25000 #11112
val_batch = 2
imageSize = 416
SAVE_LOCATION = "./data/models/model_test"
LOAD_LOCATION = "./data/models/model_best"
INTERRUPTED_LOCATION =  "./data/models/model_interrupted"
train_dir = "./data/train/"
val_dir = "./data/validation/"
log_dir = './data/logs/'

from torchvision import datasets

class ImageFolderWithPaths(datasets.ImageFolder):
    """Custom dataset that includes image file paths. Extends
    torchvision.datasets.ImageFolder
    """
    # override the __getitem__ method. this is the method dataloader calls
    def __getitem__(self, index):
        
        # this is what ImageFolder normally returns 
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        
        # the image file path
        path = self.imgs[index][0]
        
        # make a new tuple that includes original and the path
        tuple_with_path = (original_tuple + (path,))
        
        return tuple_with_path

# Load Data

In [3]:
data_transforms = transforms.Compose([transforms.Resize([imageSize,imageSize]),
                                      transforms.ToTensor()
                                     ])

# instantiate the dataset and dataloader
train_dataset = ImageFolderWithPaths(train_dir, transform=data_transforms) # our custom dataset
val_dataset = ImageFolderWithPaths(val_dir, transform=data_transforms) # our custom dataset

#loads only photos
train_dataloaders = torch.utils.data.DataLoader(train_dataset, batch_size = batchSize, shuffle=True)
val_dataloaders = torch.utils.data.DataLoader(val_dataset, batch_size = batchSize, shuffle=True)

data_factory = dataGenerator.dataGenerator(IMAGE_SIZE=imageSize)

# Training W/O Val Steps

In [4]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=4):
    # time
    since = time.time()
    best_model = None
    best_loss = math.inf
    
    outputs_dir = "data/outputs/results/" + str(datetime.now().date())
    os.makedirs(outputs_dir, exist_ok=True)
    
    # logs
    batch_log = logly.logly(True, num_epochs, num_batch)
    epoch_log = logly.logly(True, 1, num_epochs)
    batch_log_name = str(datetime.now().date()) + iteration
    epoch_log_name = "epoch - "+ str(datetime.now().date()) + iteration
    
    # model and sigmoid function
    model = model.to(device)
    sigmoid_function = nn.Sigmoid()
    
    for epoch in range(num_epochs):
        if print_loss:
            print('Epoch {}/{}'.format(epoch + 1, num_epochs), flush=True)
            
        epoch_loss = 0
        
        #Training    
        model.train()
        phase = 'train'
        for batch in range(num_batch):
            if print_loss:
                print('Batch {}/{}'.format(batch + 1, num_batch), flush=True)
            
            running_loss = 0.0
            
            #BATCH TUPLE
            inputs, labels, paths = next(iter(train_dataloaders))
            inputs = inputs.to(device)

            i = 0
            labels = torch.zeros(batchSize,NUM_CLASSES,imageSize,imageSize).to(device)

            #build ground-truth batch tensor
            for locations in paths:
                loc = locations.replace(".png", ".pt").replace("photos", "tensors")
                labels[i] = torch.load(loc)
                i += 1

            # forward
            # track history if only in train
            with torch.set_grad_enabled(phase == 'train'):
                #build input-truth batch tensor
                outputs = model(inputs)
                loss = criterion(outputs, labels) #ground truth comparison

                if phase == 'train':
                    # zero the parameter gradients
                    optimizer.zero_grad()

                    # backward + optimize 
                    loss.backward() #modifies model
                    optimizer.step() 

                # statistics
                epoch_loss += loss.item()
                running_loss = loss.item()

            batch_log.log(phase, epoch, batch, running_loss)
            
            if batch % 20 == 0:
                batch_log.write(log_dir, batch_log_name)
            
            if print_loss:
                print("loss: {}".format(running_loss), flush=True)
                print('---------------', flush=True)
        
        epoch_loss /= num_batch
        epoch_log.log(phase, 0, epoch, epoch_loss)
            
        # Validation
        model.eval()
        phase = 'val'
        val_loss = 0
        for batch in range(val_batch):
            if print_loss:
                print('Val-Batch {}/{}'.format(batch + 1, val_batch), flush=True)
            
            #BATCH TUPLE
#           inputs, labels, paths = next(iter(train_dataloaders))
            inputs, labels, paths = next(iter(val_dataloaders))
            inputs = inputs.to(device)

            i = 0
            labels = torch.zeros(batchSize,NUM_CLASSES,imageSize,imageSize).to(device)

            #build ground-truth batch tensor
            for locations in paths:
                loc = locations.replace(".png", ".pt").replace("photos", "tensors")
                labels[i] = torch.load(loc)
                i += 1
            
            with torch.set_grad_enabled(phase == 'train'):
                #build input-truth batch tensor
                outputs = model(inputs)
                loss = criterion(outputs, labels) #ground truth comparison

                # statistics
                val_loss += loss.item()
                running_loss = loss.item()

            batch_log.log(phase, epoch, batch, running_loss)
            
            if batch % 20 == 0:
                batch_log.write(log_dir, batch_log_name)
                
            if print_loss:
                print("loss: {}".format(running_loss), flush=True)
                print('---------------', flush=True)
            
            if save_masks and phase == 'val':
                outputs = sigmoid_function(outputs)
                j = 0
                for locations in paths:
                    img = Image.open(locations)
#                     num = locations.replace("./data/train/photos/", "").replace(".png", "")
#                     num = locations.replace("./data/validation/photos/", "").replace(".png", "")
                    for threshold in [0.05, .25, .5, .75]:
                        img = data_factory.showMaskOnImage(img,outputs[j], threshold)
                        img.save(outputs_dir  + "/train-" + str(num) + "-" + str(epoch) + "-t" + str(threshold) + ".png","PNG")
                    j += 1
                
        val_loss /= val_batch
        epoch_log.log(phase, 0, epoch, val_loss)
        
        epoch_log.write(log_dir, epoch_log_name)    

#             if output_tensors:
#                 return inputs, outputs, paths

        #save best copy of  model
        #if phase == 'val' and epoch_loss < best_loss:
        if epoch_loss < best_loss:
            best_loss = epoch_loss
            torch.save(model, LOAD_LOCATION)
            torch.save(model, SAVE_LOCATION)
            torch.save(model.state_dict(), LOAD_LOCATION + "-" + str(datetime.now().date()) + iteration) 
        
    #save current model
    torch.save(model,SAVE_LOCATION)
    batch_log.write(log_dir, batch_log_name)
    epoch_log.write(log_dir, epoch_log_name)
    
    
    time_elapsed = time.time() - since
    print('Training complete in {:.1f}m {:.1f}s'.format(time_elapsed // 60, time_elapsed % 60), flush=True)
    
    return model

# Load Pretrained Model Weights

In [5]:
from unet_models import *

#imports related to UNet
if newTraining:
    model = MyNet1(num_classes=1, num_filters=32, pretrained=True, is_deconv=True)
    print('Single convolution model, no weights', flush=True)
#     model = UNet16_5(num_classes=1, num_filters=32, pretrained=True, is_deconv=True)
    
#     print('initializing model with random weights', flush=True)
#     torch.nn.init.xavier_uniform_(next(model.center.children())[1].weight)
#     torch.nn.init.xavier_uniform_(next(model.dec5.children())[1].weight)
#     torch.nn.init.xavier_uniform_(next(model.dec4.children())[1].weight)
#     torch.nn.init.xavier_uniform_(next(model.dec3.children())[1].weight)
#     torch.nn.init.xavier_uniform_(next(model.dec2.children())[1].weight)
#     torch.nn.init.xavier_uniform_(next(model.dec1.children()).weight)
#     torch.nn.init.xavier_uniform_(model.final.weight)
                           
else:
    print("loading weights from", LOAD_LOCATION, flush=True)
    #model = torch.load(SAVE_LOCATION)
    model = torch.load(LOAD_LOCATION)
    #model.load_state_dict(best_model_wts)

criterion = torch.nn.BCEWithLogitsLoss()
#criterion = DICELossMultiClass()
#criterion = IOU_BCELoss()

#Observe adjustments in learning rate
optimizer_ft = optim.Adam(model.parameters(), lr=0.15, weight_decay=0, amsgrad=False, eps=0.1)

# Osscilate between high and low learning rates over time
exp_lr_scheduler = lr_scheduler.CosineAnnealingLR(optimizer_ft, T_max=EPOCHS,eta_min=0.001)  

Single convolution model, no weights


# Training and Results

In [6]:
try:
    model = train_model(model, criterion, optimizer_ft, exp_lr_scheduler,num_epochs=EPOCHS)
    
except KeyboardInterrupt:
    torch.save(model, INTERRUPTED_LOCATION)
    print('Saved interrupt', flush=True)

Epoch 1/5
Batch 1/2000
loss: 0.6933138370513916
---------------
Batch 2/2000
loss: 0.6925545334815979
---------------
Batch 3/2000
loss: 0.69579017162323
---------------
Batch 4/2000
loss: 0.6930902004241943
---------------
Batch 5/2000
loss: 0.6907747983932495
---------------
Batch 6/2000
loss: 0.6952244639396667
---------------
Batch 7/2000
loss: 0.6928170323371887
---------------
Batch 8/2000
loss: 0.6940486431121826
---------------
Batch 9/2000
loss: 0.6937956809997559
---------------
Batch 10/2000
loss: 0.691780686378479
---------------
Batch 11/2000
loss: 0.6916356086730957
---------------
Batch 12/2000
loss: 0.6957581639289856
---------------
Batch 13/2000
loss: 0.6913386583328247
---------------
Batch 14/2000
loss: 0.6896032094955444
---------------
Batch 15/2000
loss: 0.6914195418357849
---------------
Batch 16/2000
loss: 0.6919506788253784
---------------
Batch 17/2000
loss: 0.691916823387146
---------------
Batch 18/2000
loss: 0.6957119107246399
---------------
Batch 19/2000

---------------
Batch 150/2000
loss: 0.6892144680023193
---------------
Batch 151/2000
loss: 0.6903494000434875
---------------
Batch 152/2000
loss: 0.6906653642654419
---------------
Batch 153/2000
loss: 0.6925756335258484
---------------
Batch 154/2000
loss: 0.6897357106208801
---------------
Batch 155/2000
loss: 0.6914041638374329
---------------
Batch 156/2000
loss: 0.6898143887519836
---------------
Batch 157/2000
loss: 0.6870532035827637
---------------
Batch 158/2000
loss: 0.6901007890701294
---------------
Batch 159/2000
loss: 0.690382182598114
---------------
Batch 160/2000
loss: 0.692825198173523
---------------
Batch 161/2000
loss: 0.6911234259605408
---------------
Batch 162/2000
loss: 0.6914370059967041
---------------
Batch 163/2000
loss: 0.6928021311759949
---------------
Batch 164/2000
loss: 0.6903325319290161
---------------
Batch 165/2000
loss: 0.6932182312011719
---------------
Batch 166/2000
loss: 0.6925818920135498
---------------
Batch 167/2000
loss: 0.69214349985

---------------
Batch 297/2000
loss: 0.6916281580924988
---------------
Batch 298/2000
loss: 0.6916444897651672
---------------
Batch 299/2000
loss: 0.6929091215133667
---------------
Batch 300/2000
loss: 0.691268265247345
---------------
Batch 301/2000
loss: 0.691251277923584
---------------
Batch 302/2000
loss: 0.6915830969810486
---------------
Batch 303/2000
loss: 0.6908864378929138
---------------
Batch 304/2000
loss: 0.6924152970314026
---------------
Batch 305/2000
loss: 0.6911778450012207
---------------
Batch 306/2000
loss: 0.6913202404975891
---------------
Batch 307/2000
loss: 0.6899601221084595
---------------
Batch 308/2000
loss: 0.6893897652626038
---------------
Batch 309/2000
loss: 0.6899720430374146
---------------
Batch 310/2000
loss: 0.6915479302406311
---------------
Batch 311/2000
loss: 0.689721941947937
---------------
Batch 312/2000
loss: 0.6935716271400452
---------------
Batch 313/2000
loss: 0.6930205821990967
---------------
Batch 314/2000
loss: 0.689731359481

---------------
Batch 444/2000
loss: 0.6905979514122009
---------------
Batch 445/2000
loss: 0.6900048851966858
---------------
Batch 446/2000
loss: 0.6915522813796997
---------------
Batch 447/2000
loss: 0.6919156908988953
---------------
Batch 448/2000
loss: 0.6898019313812256
---------------
Batch 449/2000
loss: 0.6913827061653137
---------------
Batch 450/2000
loss: 0.6923584342002869
---------------
Batch 451/2000
loss: 0.6913497447967529
---------------
Batch 452/2000
loss: 0.6917938590049744
---------------
Batch 453/2000
loss: 0.6901151537895203
---------------
Batch 454/2000
loss: 0.6901060938835144
---------------
Batch 455/2000
loss: 0.6913232207298279
---------------
Batch 456/2000
loss: 0.6927871704101562
---------------
Batch 457/2000
loss: 0.6910082101821899
---------------
Batch 458/2000
loss: 0.6927246451377869
---------------
Batch 459/2000
loss: 0.6918464303016663
---------------
Batch 460/2000
loss: 0.6909680366516113
---------------
Batch 461/2000
loss: 0.691636562

loss: 0.6921239495277405
---------------
Batch 591/2000
loss: 0.6924373507499695
---------------
Batch 592/2000
loss: 0.6926206350326538
---------------
Batch 593/2000
loss: 0.6912916302680969
---------------
Batch 594/2000
loss: 0.6905113458633423
---------------
Batch 595/2000
loss: 0.6912184953689575
---------------
Batch 596/2000
loss: 0.6903506517410278
---------------
Batch 597/2000
loss: 0.6902198195457458
---------------
Batch 598/2000
loss: 0.6917500495910645
---------------
Batch 599/2000
loss: 0.6895854473114014
---------------
Batch 600/2000
loss: 0.6902651190757751
---------------
Batch 601/2000
loss: 0.6939019560813904
---------------
Batch 602/2000
loss: 0.6888133883476257
---------------
Batch 603/2000
loss: 0.6899014115333557
---------------
Batch 604/2000
loss: 0.6904004812240601
---------------
Batch 605/2000
loss: 0.69527667760849
---------------
Batch 606/2000
loss: 0.6907027363777161
---------------
Batch 607/2000
loss: 0.6897088885307312
---------------
Batch 608

...

loss: 0.6931471824645996
---------------
Batch 1312/2000
loss: 0.6931471824645996
---------------
Batch 1313/2000
loss: 0.6931471824645996
---------------
Batch 1314/2000
loss: 0.6931471824645996
---------------
Batch 1315/2000
loss: 0.6931471824645996
---------------
Batch 1316/2000
loss: 0.6931471824645996
---------------
Batch 1317/2000
loss: 0.6931471824645996
---------------
Batch 1318/2000
loss: 0.6931471824645996
---------------
Batch 1319/2000
loss: 0.6931471824645996
---------------
Batch 1320/2000
loss: 0.6931471824645996
---------------
Batch 1321/2000
loss: 0.6931471824645996
---------------
Batch 1322/2000
loss: 0.6931471824645996
---------------
Batch 1323/2000
loss: 0.6931471824645996
---------------
Batch 1324/2000
loss: 0.6931471824645996
---------------
Batch 1325/2000
loss: 0.6931471824645996
---------------
Batch 1326/2000
loss: 0.6931471824645996
---------------
Batch 1327/2000
loss: 0.6931471824645996
---------------
Batch 1328/2000
loss: 0.6931471824645996
------

loss: 0.6931471824645996
---------------
Batch 1456/2000
loss: 0.6931471824645996
---------------
Batch 1457/2000
loss: 0.6931471824645996
---------------
Batch 1458/2000
loss: 0.6931471824645996
---------------
Batch 1459/2000
loss: 0.6931471824645996
---------------
Batch 1460/2000
loss: 0.6931471824645996
---------------
Batch 1461/2000
loss: 0.6931471824645996
---------------
Batch 1462/2000
loss: 0.6931471824645996
---------------
Batch 1463/2000
loss: 0.6931471824645996
---------------
Batch 1464/2000
loss: 0.6931471824645996
---------------
Batch 1465/2000
loss: 0.6931471824645996
---------------
Batch 1466/2000
loss: 0.6931471824645996
---------------
Batch 1467/2000
loss: 0.6931471824645996
---------------
Batch 1468/2000
loss: 0.6931471824645996
---------------
Batch 1469/2000
loss: 0.6931471824645996
---------------
Batch 1470/2000
loss: 0.6931471824645996
---------------
Batch 1471/2000
loss: 0.6931471824645996
---------------
Batch 1472/2000
loss: 0.6931471824645996
------

loss: 0.6931471824645996
---------------
Batch 1600/2000
loss: 0.6931471824645996
---------------
Batch 1601/2000
loss: 0.6931471824645996
---------------
Batch 1602/2000
loss: 0.6931471824645996
---------------
Batch 1603/2000
loss: 0.6931471824645996
---------------
Batch 1604/2000
loss: 0.6931471824645996
---------------
Batch 1605/2000
loss: 0.6931471824645996
---------------
Batch 1606/2000
loss: 0.6931471824645996
---------------
Batch 1607/2000
loss: 0.6931471824645996
---------------
Batch 1608/2000
loss: 0.6931471824645996
---------------
Batch 1609/2000
loss: 0.6931471824645996
---------------
Batch 1610/2000
loss: 0.6931471824645996
---------------
Batch 1611/2000
loss: 0.6931471824645996
---------------
Batch 1612/2000
loss: 0.6931471824645996
---------------
Batch 1613/2000
loss: 0.6931471824645996
---------------
Batch 1614/2000
loss: 0.6931471824645996
---------------
Batch 1615/2000
loss: 0.6931471824645996
---------------
Batch 1616/2000
loss: 0.6931471824645996
------

loss: 0.6931471824645996
---------------
Batch 1744/2000
loss: 0.6931471824645996
---------------
Batch 1745/2000
loss: 0.6931471824645996
---------------
Batch 1746/2000
loss: 0.6931471824645996
---------------
Batch 1747/2000
loss: 0.6931471824645996
---------------
Batch 1748/2000
loss: 0.6931471824645996
---------------
Batch 1749/2000
loss: 0.6931471824645996
---------------
Batch 1750/2000
loss: 0.6931471824645996
---------------
Batch 1751/2000
loss: 0.6931471824645996
---------------
Batch 1752/2000
loss: 0.6931471824645996
---------------
Batch 1753/2000
loss: 0.6931471824645996
---------------
Batch 1754/2000
loss: 0.6931471824645996
---------------
Batch 1755/2000
loss: 0.6931471824645996
---------------
Batch 1756/2000
loss: 0.6931471824645996
---------------
Batch 1757/2000
loss: 0.6931471824645996
---------------
Batch 1758/2000
loss: 0.6931471824645996
---------------
Batch 1759/2000
loss: 0.6931471824645996
---------------
Batch 1760/2000
loss: 0.6931471824645996
------

loss: 0.6931471824645996
---------------
Batch 1888/2000
loss: 0.6931471824645996
---------------
Batch 1889/2000
loss: 0.6931471824645996
---------------
Batch 1890/2000
loss: 0.6931471824645996
---------------
Batch 1891/2000
loss: 0.6931471824645996
---------------
Batch 1892/2000
loss: 0.6931471824645996
---------------
Batch 1893/2000
loss: 0.6931471824645996
---------------
Batch 1894/2000
loss: 0.6931471824645996
---------------
Batch 1895/2000
loss: 0.6931471824645996
---------------
Batch 1896/2000
loss: 0.6931471824645996
---------------
Batch 1897/2000
loss: 0.6931471824645996
---------------
Batch 1898/2000
loss: 0.6931471824645996
---------------
Batch 1899/2000
loss: 0.6931471824645996
---------------
Batch 1900/2000
loss: 0.6931471824645996
---------------
Batch 1901/2000
loss: 0.6931471824645996
---------------
Batch 1902/2000
loss: 0.6931471824645996
---------------
Batch 1903/2000
loss: 0.6931471824645996
---------------
Batch 1904/2000
loss: 0.6931471824645996
------

# Testing Outputs

In [10]:
try:
    inputs, outputs, paths = train_model(model, criterion, optimizer_ft, exp_lr_scheduler,num_epochs=EPOCHS)
    #batch = 
except KeyboardInterrupt:
    torch.save(model, INTERRUPTED_LOCATION)
    print('Saved interrupt', flush=True)

# for tensor in labels:
#     #tensor = labels[0]
#     tensor_max_value = torch.max(tensor)
#     print(tensor_max_value)
#     print(type(tensor_max_value.item()))
    

In [8]:
#save state model as model
model = UNet16_5(num_classes=1, num_filters=32, pretrained=True, is_deconv=True)
model.load_state_dict(torch.load(LOAD_LOCATION+'_test-2019-04-23-3'))
torch.save(model, LOAD_LOCATION)